In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load Chattanooga APC Data 

This data comes from Teams. You can find it at: General > covid-19 > Datasets > CARTA-DATA > CARTA-APC > **chattanooga_apc_jan20_through_jun20.csv**

**Note:** A new version of chattanooga_apc_jan20_through_jun20.csv was uploaded to Teams on 2020-08-26. Make sure you are using the correct version.

In [ ]:
# read in APC data (make sure to change the file path below so it leads to where you've stored the dataset)
apc_df = pd.read_csv('chattanooga_apc_jan20_through_jun20.csv', index_col=0)

In [ ]:
# APC dataset should have 3,471,268 entries and 72 columns
apc_df.shape

In [ ]:
apc_df.head(2)

## Load in Chattanooga GTFS from GitHub

This data comes from our GitHub repository: https://github.com/hdemma/transit-hub/tree/develop/data-connect/mongo-connect/data-fusion/carta-data-join/data/GTFS. The particular txt files used here are located in Teams at: General > covid-19 > Datasets > CARTA-DATA > CARTA GTFS static. 

**Note #1:** The gtfs_may_2020 folder in GitHub is actually an update that went into effect in April 2020. GTFS was updated later.

**Note #2:** This GTFS feed is different than the one in TransitFeeds (https://transitfeeds.com/p/chattanooga-area-regional-transportation-authority/360). Do NOT use the one in TransitFeeds as it is not accurate.


In [2]:
# This feed went into effect sometime after 2019-08-18
aug19_trips_df = pd.read_csv('carta_gtfs_august_2019_trips.txt')
aug19_stops_df = pd.read_csv('carta_gtfs_august_2019_stops.txt')
aug19_stop_times_df = pd.read_csv('carta_gtfs_august_2019_stop_times.txt')

aug19_gtfs_df = aug19_trips_df.merge(aug19_stop_times_df)
aug19_gtfs_df = aug19_gtfs_df.merge(aug19_stops_df)
aug19_gtfs_df['gtfs_start_date'] = '2019-08-18' # add gtfs_start_date so it can be joined with APC data

aug19_gtfs_df.head(2)

In [ ]:
# This feed went into effect sometime after 2020-04-13
apr20_trips_df = pd.read_csv('carta_gtfs_may_2020_trips.txt')
apr20_stops_df = pd.read_csv('carta_gtfs_may_2020_stops.txt')
apr20_stop_times_df = pd.read_csv('carta_gtfs_may_2020_stop_times.txt')

apr20_gtfs_df = apr20_trips_df.merge(apr20_stop_times_df)
apr20_gtfs_df = apr20_gtfs_df.merge(apr20_stops_df)
apr20_gtfs_df['gtfs_start_date'] = '2020-04-13' # add gtfs_start_date so it can be joined with APC data

apr20_gtfs_df.head(2)

In [ ]:
# combine gtfs feeds into single df
gtfs_df = aug19_gtfs_df.append(apr20_gtfs_df)

In [ ]:
gtfs_df.head(2)

### Create a GTFS trip_key field

The software that generates GTFS feeds for CARTA appends 3 digits to the end of the APC TRIP_KEY to create a GTFS trip_id. In order to join these two datasets, we need to create a new column called trip_key in the GTFS dataset that **removes the last 3 digits of trip_id**. 

In [ ]:
gtfs_df['trip_id'] = gtfs_df['trip_id'].astype(str)
gtfs_df['trip_key'] = gtfs_df['trip_id'].str.slice(0, -3)

In [ ]:
gtfs_df.head(2)

### Check to see if trip_ids are repeated between these 2 GTFS feed updates

If not, then we will not have to match the APC data and GTFS data on gtfs_start_date field like we did for Nashville.

In [ ]:
# there are 3158 unique trip_ids (and trip_keys) across both GTFS feeds
print("unique trip_ids:", gtfs_df['trip_id'].nunique())
print("unique trip_keys:", gtfs_df['trip_key'].nunique())

In [ ]:
# get unique pairs of trip_id and gtfs_start_date
# if all trip_ids are unique to a GTFS start date, then we should also get back 3158 rows
gtfs_df2 = gtfs_df.drop_duplicates(['trip_key', 'gtfs_start_date'])
gtfs_df2.shape[0]

In [ ]:
# confirm results by counting the number of appearances for each trip_id in unique pairs of trip_id and gtfs_start_date
gtfs_df2 = gtfs_df2.groupby(['trip_key']).size().reset_index()
gtfs_df2.columns = ['trip_key', 'num_appearances']
gtfs_df2.sort_values('num_appearances', ascending=False).head()

*This shows that trip_keys are not reused between GTFS these 2 feed updates, so this will be a safe way to match the two datasets.*

#### Compare to Nashville GTFS

Nashville GTFS is known to have trip_id repeats between GTFS feeds. This data comes from Teams under General > covid-19 > Datasets > WeGO-Data > data-used-for-analysis > Nashville GTFS (static and realtime) > **gtfs.csv**

In [ ]:
nashville_gtfs_df = pd.read_csv('gtfs.csv', index_col=0)
nashville_gtfs_df.head(2)

In [ ]:
# 11,636 unique trips
nashville_gtfs_df['trip_id'].nunique()

In [ ]:
# since all trip_ids are NOT unique to a GTFS start date, we are getting back more than 11,636 rows
nashville_gtfs_df2 = nashville_gtfs_df.drop_duplicates(['trip_id', 'gtfs_start_date'])
nashville_gtfs_df2.shape[0]

In [ ]:
# confirm results by counting the number of appearances for each trip_id in unique pairs of trip_id and gtfs_start_date
nashville_gtfs_df2 = nashville_gtfs_df2.groupby(['trip_id']).size().reset_index()
nashville_gtfs_df2.columns = ['trip_id', 'num_appearances']
nashville_gtfs_df2.sort_values('num_appearances', ascending=False).head()

## Join APC and GTFS datasets

### Before we join:
Let's see if:
1. there are trip_keys that exist in the APC data that are not in the GTFS data
2. there are stop_ids that exist in the APC data that are not in the GTFS data


In [ ]:
# convert to str; data types need to match for join to work
gtfs_df['stop_id'] = gtfs_df['stop_id'].astype(str)
gtfs_df['trip_key'] = gtfs_df['trip_key'].astype(str)

apc_df['STOP_ID'] = apc_df['STOP_ID'].astype(str)
apc_df['TRIP_KEY'] = apc_df['TRIP_KEY'].astype(str)

In [ ]:
# 1. check trip keys
apc_trip_keys = set(apc_df['TRIP_KEY'].unique())
len(apc_trip_keys)

In [ ]:
gtfs_trip_keys = set(gtfs_df['trip_key'].unique())
len(gtfs_trip_keys)

In [ ]:
# the following trips_keys are not found in GTFS
apc_trip_keys_not_in_gtfs = apc_trip_keys - gtfs_trip_keys
len(apc_trip_keys_not_in_gtfs)

The following trip_keys are found in the APC dataset but not in GTFS:
{'134062',
 '134067',
 '134074',
 '134079',
 '134518',
 '134530',
 '135676',
 '135678',
 '135680',
 '135683',
 '135684',
 '135686',
 '135689',
 '135721',
 '135723',
 '135725',
 '135728',
 '135729',
 '135731',
 '135734',
 '135851',
 '135864',
 '135870',
 '135875',
 '135880',
 '135886',
 '135945',
 '135953',
 '135963',
 '135964',
 '135973',
 '135981',
 '135994',
 '136471',
 '136475',
 '137339',
 '137351',
 '137353',
 '137474',
 '137477',
 '137478',
 '137502',
 '137506',
 '137507'}

In [ ]:
# 2. check stop_ids
apc_stop_ids = set(apc_df['STOP_ID'].unique())
len(apc_stop_ids)

In [ ]:
gtfs_stop_ids = set(gtfs_df['stop_id'].unique())
len(gtfs_stop_ids)

In [ ]:
apc_stop_ids_not_in_gtfs = apc_stop_ids - gtfs_stop_ids
len(apc_stop_ids_not_in_gtfs)

The following stop_ids are found in the APC dataset but not in GTFS: {'100004',
 '100008',
 '100010',
 '100012',
 '100014',
 '100016',
 '100018',
 '100019',
 '100023',
 '100026',
 '100027',
 '100028',
 '100029',
 '100030',
 '100035',
 '100036',
 '100037',
 '100041',
 '100043',
 '100044',
 '100045',
 '100047',
 '100048',
 '100051',
 '100052',
 '100054',
 '100058',
 '100059',
 '100061',
 '100062',
 '100066',
 '100069',
 '100071',
 '100073',
 '100074',
 '100075',
 '100077',
 '100079',
 '100080',
 '100081',
 '100088',
 '100098',
 '100105',
 '100108',
 '100127',
 '100133',
 '100143',
 '100144',
 '100145',
 '100149',
 '100151',
 '100173',
 '100181',
 '100183',
 '100186',
 '100189',
 '100190',
 '100193',
 '100195',
 '100196',
 '100198',
 '100199',
 '100200',
 '100204',
 '100205',
 '100210',
 '100214',
 '100216',
 '100217',
 '100218',
 '100221',
 '100222',
 '100223',
 '100226',
 '100227',
 '100229',
 '100230',
 '100231',
 '100232',
 '100235',
 '100244',
 '100246',
 '100249',
 '100251',
 '100252',
 '100253',
 '100254',
 '100255',
 '100256',
 '100257',
 '100258',
 '1428',
 '1449',
 '1591',
 '1611',
 '403',
 '406',
 '411',
 '416',
 '417',
 '418',
 '419'}

**Even though there are some trip_keys and stop_ids that will not be matched, let us continue on with the data join.**

## There are two ways to join these datasets:

#### Method #1: join on trip_key, stop_id, AND gtfs_start_date
- Use this method when trip_ids are repeated across GTFS feed updates (for example, in Nashville GTFS)

#### Method #2: join on trip_key and stop_id
- Use this method when you are sure that trip_ids do not repeat across GTFS feed updates.

**The following notebook outlines the steps for both methods, but the final dataset used in the dashboard is produced using method #2**

From the earlier section entitled "Check to see if trip_ids are repeated between these 2 GTFS feed updates", it looks like trip_ids may not be repeated. However, we are still waiting on confirmation from Clever. This is why both methods are included. 

### Method #1: 

In [ ]:
# 1. APC: Determine the date on which the APC data was taken

# convert SIGNUP_DATE to datetime obj
apc_df['DATE'] = pd.to_datetime(apc_df['SURVEY_DATE'])

In [ ]:
# 2. APC: Find the date on which the GTFS feed in service at the time went into effect

# create gtfs_start_date field
def get_gtfs_start_date(date):
    if date < datetime.datetime(2020, 4, 14):
        return '2019-08-18'
    return '2020-04-14'

apc_df['GTFS_START_DATE'] = apc_df.apply(lambda row: get_gtfs_start_date(row['DATE']), axis=1)

In [ ]:
# convert to str; data types need to match for join to work
gtfs_df['stop_id'] = gtfs_df['stop_id'].astype(str)
gtfs_df['trip_key'] = gtfs_df['trip_key'].astype(str)
gtfs_df['gtfs_start_date'] = gtfs_df['gtfs_start_date'].astype(str)

apc_df['STOP_ID'] = apc_df['STOP_ID'].astype(str)
apc_df['TRIP_KEY'] = apc_df['TRIP_KEY'].astype(str)
apc_df['GTFS_START_DATE'] = apc_df['GTFS_START_DATE'].astype(str)

In [ ]:
#try join
test = apc_df.merge(gtfs_df, left_on=['TRIP_KEY', 'STOP_ID', 'GTFS_START_DATE'], right_on=['trip_key', 'stop_id', 'gtfs_start_date'], how='left')
test.head(2)

In [ ]:
# we should have the same numner of rows in apc_df as test_df, but we're coming up with extra for some reasons
test.shape[0] - apc_df.shape[0] 

### Method #2: 

In [ ]:
#try join
test = apc_df.merge(gtfs_df, left_on=['TRIP_KEY', 'STOP_ID'], right_on=['trip_key', 'stop_id'], how='left')
test.head(2)

In [ ]:
# if all was successful, we should have the same numner of rows in apc_df as test_df
test.shape[0] - apc_df.shape[0]

## Getting Rid of Extra Rows 

We are getting 204 extra rows in our join. This is because there are duplicate combinations of trip_key and stop_id in GTFS. In other words, on a particular trip, a single stop_id can appear more than once. We need to deal with these duplicates.

In [ ]:
# look at all duplicates
gtfs_df.loc[gtfs_df.duplicated(subset=['trip_id', 'stop_id'], keep=False)]

#### It looks like all the extra rows result from duplicate stop_ids on trip_id: 138668020

To remove these duplicates, we want to: 
- keep the FIRST occurence of date, trip_id = 138668020, gtfs_start_date, 
stop_id when stop_sequence = 2, 3, 4
- keep the LAST occurence of date, trip_id = 138668020, gtfs_start_date, stop_id when stop_sequence = 32, 33, 34



We will accomplish this by: 

1. divide the dataframe into 2 parts
  - where trip_id == 138668020
  - where trip_id != 138668020

2. for the dataframe where trip_id == 138668020
  - keep the FIRST occurence of date, trip_id = 138668020, gtfs_start_date, 
stop_id when stop_sequence = 2, 3, 4
  - keep the LAST occurence of date, trip_id = 138668020, gtfs_start_date, stop_id when stop_sequence = 32, 33, 34

3. combine the two dataframes back together

In [ ]:
# 1. divide the dataframe into 2 parts
no_duplicates = test.loc[test['trip_id'] != '138668020']
duplicates = test.loc[test['trip_id'] == '138668020']

# check how many entries are in the duplicates dataframe (3400)
duplicates.shape[0] 

In [ ]:
# 2. For the dataframe where trip_id == 138668020:

# keep FIRST occurence of date, trip_id = 138668020, gtfs_start_date, stop_id when stop_sequence = 2, 3, 4
keep_first = duplicates.loc[duplicates['stop_sequence'].isin([2, 3, 4])]
keep_first = keep_first.drop_duplicates(subset=['SURVEY_DATE','stop_id'], keep='first')

# keep LAST occurence of date, trip_id = 138668020, gtfs_start_date, stop_id when stop_sequence = 32, 33, 34
keep_last = duplicates.loc[duplicates['stop_sequence'].isin([32, 33, 34])]
keep_last = keep_last.drop_duplicates(subset=['SURVEY_DATE','stop_id'],keep='last')

# for all other entries where trip_id == 138668020, do nothing
keep_all = duplicates.loc[~duplicates['stop_sequence'].isin([2, 3, 4, 32, 33, 34])]

# merge everything together
duplicates_removed = keep_first.append(keep_last)
duplicates_removed = duplicates_removed.append(keep_all)

In [ ]:
# we should have removed 204 entries
duplicates.shape[0] - duplicates_removed.shape[0]

In [ ]:
# 3. combine the 2 dataframes back together
test2 = no_duplicates.append(duplicates_removed)

In [ ]:
# confirm that there are 3,471,268 rows
test2.shape[0]

In [ ]:
# confirm that we dropped the right entries (stop_sequence should increase sequentially)
test2 = test2.sort_index()
test2.loc[test2['trip_id'] == '138668020'].head(50)[['SURVEY_DATE', 'trip_id', 'stop_sequence']]

In [ ]:
test2 = test2.reset_index(drop=True)
test2.tail(2) # index of last entry should be 3,471,267

## Load in RideCheck Stops Data 

Since GTFS wasn't able to provide the lat/lon of all stops, there is additional information that we can pull in straight from RideCheck (the software that produces the APC data). All stop_ids should be present, so we don't have to worry about missing data.

This data comes from Teams. You can find it under General > covid-19 > Datasets > CARTA-DATA > **STOPS.xlsx**.

In [ ]:
apc_stops_df = pd.read_excel('STOPS.xlsx')[['STOP_ID', 'MAIN_STREET', 'CROSS_STREET', 'LATITUDE', 'LONGITUDE']]

In [ ]:
apc_stops_df.head(2)

## Join RideCheck Stop Data

We perform a LEFT join with RideCheck stop data (from STOPS.xlsx) on STOP_ID. There should be no null values.

In [ ]:
test2['STOP_ID'] = test2['STOP_ID'].astype(str)
apc_stops_df['STOP_ID'] = apc_stops_df['STOP_ID'].astype(str)

test3 = test2.merge(apc_stops_df, left_on='STOP_ID', right_on='STOP_ID', how='left')

In [ ]:
# check for null values
test3.loc[test3['LATITUDE'].isnull()].shape[0]

In [ ]:
# check that no rows were added
test3.shape[0] - test2.shape[0]

## Checking Joined Dataset

How many APC entries could be matched to GTFS entries? Where did trip_key and stop_id not match up? 
- If GTFS information could not be matched to the APC entry, we will have a NULL value for trip_id

In [ ]:
# check where stop_id is null (could not join with GTFS)
missing_gtfs = test3.loc[test3['stop_id'].isnull()]
missing_gtfs.shape[0] # 316,821

In [ ]:
# ~9% rows are missing GTFS data
missing_gtfs.shape[0] / test3.shape[0]

In [ ]:
# are some routes more likely to be missing GTFS than others?
routes1 = pd.DataFrame(missing_gtfs.ROUTE_NUMBER.value_counts())
routes2 = pd.DataFrame(test3.ROUTE_NUMBER.value_counts())
routes = routes1.merge(routes2, left_index=True, right_index=True).reset_index()
routes.columns = ['Route', 'Missing GTFS', 'Total']
routes['% Missing GTFS'] = routes['Missing GTFS'] / routes['Total'] * 100
routes = routes.sort_values('% Missing GTFS', ascending=False)
routes

In [ ]:
# are some dates more likely to be missing GTFS than others?
dates1 = pd.DataFrame(missing_gtfs.SURVEY_DATE.value_counts())
dates2 = pd.DataFrame(test3.SURVEY_DATE.value_counts())
dates = dates1.merge(dates2, left_index=True, right_index=True).reset_index()
dates.columns = ['Date', 'Missing GTFS', 'Total']
dates['% Missing GTFS'] = dates['Missing GTFS'] / dates['Total'] * 100
dates = dates.sort_values('% Missing GTFS', ascending=False)
dates.head()

In [ ]:
# check by month
dates['Month'] = pd.to_datetime(dates['Date']).dt.month
month = dates.groupby('Month').sum()
month['% Missing GTFS'] = month['Missing GTFS'] / month['Total'] * 100
month

In [ ]:
# are some days more likely to be missing GTFS than others?
day_of_week1 = pd.DataFrame(missing_gtfs.SERVICE_PERIOD.value_counts())
day_of_week2 = pd.DataFrame(test3.SERVICE_PERIOD.value_counts())
day_of_week = day_of_week1.merge(day_of_week2, left_index=True, right_index=True).reset_index()
day_of_week.columns = ['Day', 'Missing GTFS', 'Total']
day_of_week['% Missing GTFS'] = day_of_week['Missing GTFS'] / day_of_week['Total'] * 100
day_of_week = day_of_week.sort_values('Day')
day_of_week

In [ ]:
# are some days more likely to be missing GTFS than others?
time_of_day1 = pd.DataFrame(missing_gtfs.TIME_PERIOD.value_counts())
time_of_day2 = pd.DataFrame(test3.TIME_PERIOD.value_counts())
time_of_day = time_of_day1.merge(time_of_day2, left_index=True, right_index=True).reset_index()
time_of_day.columns = ['Time of Day', 'Missing GTFS', 'Total']
time_of_day['% Missing GTFS'] = time_of_day['Missing GTFS'] / time_of_day['Total'] * 100
time_of_day = time_of_day.sort_values('Time of Day')
time_of_day

## Save Final Joined Dataset

In [ ]:
test3.to_csv('chattanooga_bus_occupancy_jan20_through_jun20.csv')

## Create Dataset Used for Chattanooga Dashboard

A few more things need to be done for the dataset to be used in the Chattanooga occupancy dashboard: 

Dashboard dataset should have the following columns: 

'trip_id', 'arrival_time', 'stop_id', 'stop_sequence', 'stop_name', 'stop_lat', 'stop_lon', 'route_id',
'direction_id', 'date', 'board_count',
'alight_count', 'occupancy', 'direction_desc',
'date_time', 'trip_start_time', 'trip_name', 'day_of_week',
'service_period'

1. Drop rows with null values (where GTFS did not get matched)

2. Calculate additional fields used in the dataset if they do not already exist  

3. Change column names (column names used in the dashboard should be the same between Nashville and Chattanooga)

In [ ]:
# change column names
chattanooga_dashboard_df = test3[['trip_id', 'arrival_time', 'stop_id', 'stop_sequence', 'stop_name',
                                    'stop_lat', 'stop_lon', 'route_id', 'direction_id', 'SURVEY_DATE', 
                                    'PASSENGERS_ON', 'PASSENGERS_OFF', 'PASSENGERS_IN', 'DIRECTION_NAME', 'SERVICE_PERIOD']]
print("num rows:", chattanooga_dashboard_df.shape[0])
chattanooga_dashboard_df.head(2)

In [ ]:
# drop null values
chattanooga_dashboard_df = chattanooga_dashboard_df.dropna()
print("num rows after null values are dropped:", chattanooga_dashboard_df.shape[0])

In [ ]:
# calculate date field and drop SURVEY_DATE
chattanooga_dashboard_df['date'] = pd.to_datetime(chattanooga_dashboard_df['SURVEY_DATE'])
chattanooga_dashboard_df['date'].sample(5)

In [ ]:
# drop SURVEY_DATE (will use 'date' field in dashboard instead)
chattanooga_dashboard_df = chattanooga_dashboard_df.drop(columns=['SURVEY_DATE'])

In [ ]:
# add date_time field
chattanooga_dashboard_df['date'] = chattanooga_dashboard_df['date'].astype(str)
chattanooga_dashboard_df['date_time'] = chattanooga_dashboard_df['date'] + " " + chattanooga_dashboard_df['arrival_time']
chattanooga_dashboard_df['date_time'].sample(5)

In [ ]:
chattanooga_dashboard_df.head(2)

In [ ]:
# add trip_start_time
sorted_by_time = chattanooga_dashboard_df.sort_values('arrival_time')
trip_start_time = chattanooga_dashboard_df.drop_duplicates('trip_id', keep='first')
trip_start_time = trip_start_time[['trip_id', 'arrival_time']]
trip_start_time.columns = ['trip_id', 'trip_start_time']
chattanooga_dashboard_df = chattanooga_dashboard_df.merge(trip_start_time, on='trip_id', how='left')

chattanooga_dashboard_df[['trip_id', 'date_time', 'trip_start_time', 'arrival_time', 'stop_sequence']].sample(5)

In [ ]:
# add trip name
chattanooga_dashboard_df['trip_name'] = chattanooga_dashboard_df['trip_start_time'] + ' (trip ID: ' + chattanooga_dashboard_df['trip_id'] + ')'
chattanooga_dashboard_df['trip_name'].sample(5)

In [ ]:
# add day of week
chattanooga_dashboard_df['date'] = pd.to_datetime(chattanooga_dashboard_df['date'])
chattanooga_dashboard_df['day_of_week'] = chattanooga_dashboard_df['date'].dt.dayofweek

In [ ]:
# check final columns
chattanooga_dashboard_df.head(2)

In [ ]:
chattanooga_dashboard_df =  chattanooga_dashboard_df.rename(columns={'PASSENGERS_ON' : 'board_count',
                                                                     'PASSENGERS_OFF' : 'alight_count',
                                                                     'PASSENGERS_IN' : 'occupancy',
                                                                     'DIRECTION_NAME' : 'direction_desc',
                                                                     'SERVICE_PERIOD' : 'service_period'})
chattanooga_dashboard_df.head(2)

## Save dashboard dataset to CSV

In [ ]:
chattanooga_dashboard_df.to_csv('chattanooga_bus_occupancy_dashboard_20200828_update.csv')